# Hacker News Posts Analysis Project

For this project, I am going to look at a data set of postings made on Hacker News. Hacker News is a website where user-submitted posts are voted and commented upon, similar to reddit, but which is extremely popular in tech and startup communities. "Show HN" posts are posts in which a user shows something to the Hacker News community, and "Ask HN" posts are posts in which a user asks the Hacker News community a question. The goal is to determine whether "Show HN" or "Ask HN" posts receive more comments and whether posts made at a certain time of day receive more comments.

## Exploratory Data Analysis

First, I will import in the data set as a list. The data set is from Kaggle and the documentation is [here](https://www.kaggle.com/hacker-news/hacker-news-posts/data#).

It should be noted that the data set I'm working with was reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments and then randomly sampling from the remaining submissions.

In [1]:
from csv import reader
opened_file = open('hacker_news.csv')
read_file = reader(opened_file)
hn = list(read_file)

print(hn[0:5])

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'], ['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']]


Since it is clear that the data set has a header, I will remove the header and assign it to its own variable in order to make the analysis easier.

In [2]:
headers = hn[0]
hn = hn[1:]
print(headers)
print(hn[0:5])
print(len(hn))

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']
[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']]
20100


The data set contains 20,100 observations of Hacker News posts. The columns in the data set are: Post ID, Post Title, URL of Linked Item, Number of Upvotes, Number of Comments, Author, and Date/Time Created.

## Analysis of Post Type

To start, I want create three separate data sets corresponding to the posts that start with "Ask HN", the posts that start with "Show HN", and all the remaining posts. I want to focus specifically on these types of posts to see whether users asking questions or users showing something to the community generates more conversation on the platform.

I will do this by first creating three blank lists, then looping over each row in the dataset corresponding to a single Hacker News post. For each row, I will take the title and change it to all lowercase to make sure that capitalization does not affect the search results. I will then check if the title starts with either "ask hn" or "show hn" using the startswith method for the string class. If so, I will append that entire row to the appropriate list, and if not, I will append it to the "Other Posts" list.

In [3]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    title = title.lower()
    if title.startswith('ask hn'):
        ask_posts.append(row)
    elif title.startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)

print(len(ask_posts))
print(len(show_posts))
print(len(other_posts))

1744
1162
17194


The result is a total of 1,744 "Ask HN" posts and a total of 1,162 "Show HN" posts.

Next, I will determine whether "Ask HN" posts or "Show HN" posts receive more comments on average. To do this, I will initialize a total "Ask HN" comments variable with a value of 0. I will then loop over each row in the "Ask HN" data set, convert the number of comments to an integer, and increment the total "Ask HN" comments variable by the number of comments that that particular post received. After looping over all the rows, I will calculate the average number of comments by dividing the total number of "Ask HN" comments by the length of the "Ask HN" data set, which tells us how many posts are in that data set. 

I will then do the same thing for the "Show HN" data set.

In [4]:
total_ask_comments = 0

for row in ask_posts:
    num_comments = int(row[4])
    total_ask_comments += num_comments
avg_ask_comments = total_ask_comments / len(ask_posts)
print(avg_ask_comments)

total_show_comments = 0

for row in show_posts:
    num_comments = int(row[4])
    total_show_comments += num_comments
avg_show_comments = total_show_comments / len(show_posts)
print(avg_show_comments)

14.038417431192661
10.31669535283993


The result shows that the "Ask HN" posts have more comments on average, with an average of 14 comments, as opposed to an average of about 10 comments on the "Show HN" posts. Therefore, I will focus on only the "Ask HN" posts going forward.

## Analysis of Post Time

I want to figure out how many "Ask HN" posts are made each hour of the day and how many comments on "Ask HN" posts are made each hour of the day. To do this, I will first create a new consolidated list that only contains the date and time that each "Ask HN" post was created and the number of comments that that post received. 

In [5]:
import datetime as dt

result_list = []

for row in ask_posts:
    created_at = row[6]
    num_comments = int(row[4])
    result_list.append([created_at, num_comments])

I now want to create two dictionaries: one that contains how many "Ask HN" posts were made each hour of the day and another that contains how many comments were made on "Ask HN" posts each hour of the day. To do this, I will first import the datetime module and create two blank dictionaries. I will loop over each row in the new consolidated data set and convert the date and time that the post was created, which is currently stored as a string, to a datetime object using the strptime method. I will then extract only the hour as a string from that datetime object using the strftime method. If the hour that that particular post was made is not yet a key in the dictionary, I will create a new entry in one dictionary with a count of 1, as well as create a new entry in the other dictionary with the number of comments that post had. If the hour that that particular post was made is already a key in the dictionaries, I will increment the count by 1 and increment the number of comments by how many comments that post had. 

In [6]:
counts_by_hour = {}
comments_by_hour= {}

for row in result_list:
    created_at = row[0]
    num_comments = row[1]
    created_at_datetime = dt.datetime.strptime(created_at, "%m/%d/%Y %H:%M")
    hour = dt.datetime.strftime(created_at_datetime, "%H")
    if hour not in counts_by_hour:
        counts_by_hour[hour] = 1
        comments_by_hour[hour] = num_comments
    else:
        counts_by_hour[hour] += 1
        comments_by_hour[hour] += num_comments

Now that I have the dictionaries showing how many posts were made each hour and how many comments were made each hour, I want calculate an average number of comments for each hour of the day. To do this, I will loop over each hour in the dictionary and calculate the number of comments divided by the number of posts at that hour. I will then create a new list using the hour as the first value and the average number of comments as the second value and append it to a blank list. The result is a new consolidated data set that contains the hour and the average number of comments across all "Ask HN" posts.

In [7]:
avg_by_hour = []

for key in comments_by_hour:
    avg_by_hour.append([key, comments_by_hour[key] / counts_by_hour[key]])
    
print(avg_by_hour)

[['19', 10.8], ['00', 8.127272727272727], ['06', 9.022727272727273], ['12', 9.41095890410959], ['23', 7.985294117647059], ['14', 13.233644859813085], ['03', 7.796296296296297], ['10', 13.440677966101696], ['05', 10.08695652173913], ['09', 5.5777777777777775], ['02', 23.810344827586206], ['22', 6.746478873239437], ['11', 11.051724137931034], ['17', 11.46], ['20', 21.525], ['08', 10.25], ['21', 16.009174311926607], ['15', 38.5948275862069], ['13', 14.741176470588234], ['18', 13.20183486238532], ['16', 16.796296296296298], ['04', 7.170212765957447], ['01', 11.383333333333333], ['07', 7.852941176470588]]


Since the output is hard to analyze in this format, I want to sort the data from highest average number of comments to lowest average number of comments to get a better idea of which hours are the best for making posts. To do this, I will loop over each row and append the columns in opposite order to a new data set. Having the average number of comments first will allow me to sort by this value. I will then use the sorted function to sort the data set in descending order. 

In [8]:
swap_avg_by_hour = []
for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])
print(swap_avg_by_hour)

sorted_swap = sorted(swap_avg_by_hour, reverse = True)

[[10.8, '19'], [8.127272727272727, '00'], [9.022727272727273, '06'], [9.41095890410959, '12'], [7.985294117647059, '23'], [13.233644859813085, '14'], [7.796296296296297, '03'], [13.440677966101696, '10'], [10.08695652173913, '05'], [5.5777777777777775, '09'], [23.810344827586206, '02'], [6.746478873239437, '22'], [11.051724137931034, '11'], [11.46, '17'], [21.525, '20'], [10.25, '08'], [16.009174311926607, '21'], [38.5948275862069, '15'], [14.741176470588234, '13'], [13.20183486238532, '18'], [16.796296296296298, '16'], [7.170212765957447, '04'], [11.383333333333333, '01'], [7.852941176470588, '07']]


Finally, I want to print the results for the top 5 hours for "Ask HN" posts in a better format that is easier to read. To do this, I will loop over the first 5 rows in the sorted list and convert the hour to an hour:minute format by converting the hour, which is currently stored as a string, to a datetime object, then converting it back to a string in the format I want using strftime. I will then create a template using {} brackets where the average number of comments and hour will be filled in. I will ensure that the average number of comments is formatted to only 2 decimal places using the .2f notation. I will then use the format method for strings to fill in the hour and average number of comments to this template and print it.

In [9]:
print("Top 5 Hours for Ask Posts Comments")

for row in sorted_swap[0:5]:
    hour = row[1]
    hour = dt.datetime.strptime(hour, "%H")
    hour = dt.datetime.strftime(hour, "%H:%M")
    average = row[0]
    template = "{}: {:.2f} average comments per post"
    print(str.format(template, hour, average))

Top 5 Hours for Ask Posts Comments
15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.80 average comments per post
21:00: 16.01 average comments per post


Based on this result, the best time to make a "Ask HN" post is at 3 pm Eastern Time. 

## Conclusion

An analysis of over 20,000 posts on Hacker News showed that posts in which the user asks the community a question generally generate more feedback and conversation than a post in which the user shows the community something. Furthermore, if a user is going to ask the Hacker News community a question, the best time to do this is 3 p.m. Eastern Time if the user wants to receive the most possible answers from the community.